In [1]:
!pip install findspark

Defaulting to user installation because normal site-packages is not writeable


In [2]:
!pip install pyspark

Defaulting to user installation because normal site-packages is not writeable


In [22]:
from pyspark import SparkContext, SparkConf

In [23]:
conf = SparkConf().setAppName("test").setMaster("local[*]")
sc = SparkContext(conf=conf)

In [24]:
data=[1,2,3,4,5]

In [25]:
distData=sc.parallelize(data, 2)

In [26]:
lines=sc.textFile("word-count/input/Alice-in-Wonderland.txt");

In [27]:
lineLengths = lines.map(lambda s: len(s))

In [28]:
totalLength = lineLengths.reduce(lambda a,b: a + b)

In [29]:
print(totalLength)

154475


In [33]:
distData.map(print)

PythonRDD[7] at RDD at PythonRDD.scala:53

In [41]:
distData.foreach(print)

3
4
5
1
2


In [43]:
coll = distData.collect()

In [44]:
for val in coll:
    print(val)

1
2
3
4
5


In [45]:
pairs = lines.map(lambda s: (s, 1))

In [46]:
counts = pairs.reduceByKey(lambda a, b: a + b)